In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from termcolor import colored
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer



from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline

# Datos Puros

In [4]:
data = pd.read_csv('./data_identificada.csv')
data.head(1)

,Netflows,First_Protocol,Second_Protocol,Third_Protocol,p1_d,p2_d,p3_d,duration,max_d,min_d,...,second_dp,third_dp,p1_ip,p2_ip,p3_ip,p1_ib,p2_ib,p3_ib,Type,Cryptocurrency
0,168,TCP,UDP,NaN,0.0,0.0,3.00375,1354.823,60.004,0.0,...,80.0,53.0,1.0,1.0,2.0,32.0,32.0,64.0,not_mine,Does not have


In [5]:
data = data.drop(['Second_Protocol', 'Third_Protocol'], axis = 1)

## Imputación de valores ausentes

In [6]:
punter = pd.concat([data['second_sp'],data['second_dp'],data['third_sp'],data['third_dp']], axis = 1)
imputer = SimpleImputer(missing_values = np.nan, strategy = "median")
values = imputer.fit_transform(punter)

In [7]:
temp = data.drop(['second_sp','second_dp','third_sp','third_dp'], axis = 1)
punter = pd.DataFrame(values, columns = punter.columns)
data = pd.concat([temp,punter], axis = 1)
data.head(2)

,Netflows,First_Protocol,p1_d,p2_d,p3_d,duration,max_d,min_d,#packets,Avg_bps,...,p3_ip,p1_ib,p2_ib,p3_ib,Type,Cryptocurrency,second_sp,second_dp,third_sp,third_dp
0,168,TCP,0.00000,0.000,3.00375,1354.823,60.004,0.000,5335,212962,...,2.0,32.00,32.0,64.0,not_mine,Does not have,80.0,80.0,53.0,53.0
1,30,TCP,148.48175,164.661,194.53325,4839.548,273.415,3.002,234,474,...,8.0,162.75,230.5,312.0,not_mine,Does not have,58704.0,1900.0,32818.0,58704.0


# Exclusión de variables con varianza próxima a cero

# Eliminar la multicolinialidad

## OneHotEncoder

In [8]:
data_categoric = data[data.select_dtypes(include=['object']).columns.to_list()]
one_hot =  OneHotEncoder(drop="first")
one_hot.fit_transform(data_categoric)

<4733x9 sparse matrix of type '<class 'numpy.float64'>'
	with 11481 stored elements in Compressed Sparse Row format>

In [9]:
one_hot.categories_

[array(['ICMP', 'ICMP6', 'TCP', 'UDP'], dtype=object),
 array(['mine', 'not_mine'], dtype=object),
 array(['Bitcash', 'Bitcoin', 'Does not have', 'Etherium', 'Litecoin',
        'Monero'], dtype=object)]

In [10]:
dataDummy = pd.get_dummies(data_categoric)
dataDummy.head()

,First_Protocol_ICMP,First_Protocol_ICMP6,First_Protocol_TCP,First_Protocol_UDP,Type_mine,Type_not_mine,Cryptocurrency_Bitcash,Cryptocurrency_Bitcoin,Cryptocurrency_Does not have,Cryptocurrency_Etherium,Cryptocurrency_Litecoin,Cryptocurrency_Monero
0,0,0,1,0,0,1,0,0,1,0,0,0
1,0,0,1,0,0,1,0,0,1,0,0,0
2,0,0,1,0,0,1,0,0,1,0,0,0
3,0,0,1,0,0,1,0,0,1,0,0,0
4,0,0,1,0,0,1,0,0,1,0,0,0


## Estandarización

In [11]:
data_numeric = data[data.select_dtypes(include=['float64', 'int64']).columns.to_list()]

In [12]:
preprocessor = ColumnTransformer([
    ('scale', StandardScaler(), data_numeric.columns), 
], remainder='passthrough')

In [13]:
values = preprocessor.fit_transform(data_numeric)
values

array([[ 0.36417233, -0.70094869, -0.80447685, ..., -0.80497468,
        -0.48368833, -0.36344266],
       [-0.35625065,  0.07866745,  0.08494276, ..., -0.73388899,
         1.23041409,  3.70378776],
       [ 0.22322001, -0.63266483, -0.24175734, ..., -0.59894335,
        -0.48227582, -0.23536003],
       ...,
       [-0.50242342,  2.1944395 ,  2.17414251, ..., -0.24394453,
        -0.48002628, -0.3363976 ],
       [-0.50242342,  0.72668554,  0.66419632, ...,  1.3982912 ,
        -0.48002628, -0.3363976 ],
       [-0.50242342,  2.44314661,  2.43006651, ..., -0.63452525,
        -0.48002628, -0.3363976 ]])

In [14]:
data_standarizada =  pd.DataFrame(values, columns = data_numeric.columns)
data_standarizada.head(1)

,Netflows,p1_d,p2_d,p3_d,duration,max_d,min_d,#packets,Avg_bps,Avg_pps,...,p1_ip,p2_ip,p3_ip,p1_ib,p2_ib,p3_ib,second_sp,second_dp,third_sp,third_dp
0,0.364172,-0.700949,-0.804477,-0.943635,-0.31695,-0.898724,-0.625504,0.37276,0.580955,0.446664,...,-0.368592,-0.401682,-0.451617,-0.424797,-0.48902,-0.572839,-0.824838,-0.804975,-0.483688,-0.363443


### Concatenación de los conjuntos de datos


In [15]:
data_p = pd.concat([data_standarizada, dataDummy], axis = 1)
data_p.columns

Index(['Netflows', 'p1_d', 'p2_d', 'p3_d', 'duration', 'max_d', 'min_d',
       '#packets', 'Avg_bps', 'Avg_pps', 'Avg_bpp', '#Bytes', '#sp', '#dp',
       'first_sp', 'first_dp', 'p1_ip', 'p2_ip', 'p3_ip', 'p1_ib', 'p2_ib',
       'p3_ib', 'second_sp', 'second_dp', 'third_sp', 'third_dp',
       'First_Protocol_ICMP', 'First_Protocol_ICMP6', 'First_Protocol_TCP',
       'First_Protocol_UDP', 'Type_mine', 'Type_not_mine',
       'Cryptocurrency_Bitcash', 'Cryptocurrency_Bitcoin',
       'Cryptocurrency_Does not have', 'Cryptocurrency_Etherium',
       'Cryptocurrency_Litecoin', 'Cryptocurrency_Monero'],
      dtype='object')

In [16]:
data_p.to_csv('dataA.csv', index=False)

# Data No Pura

In [17]:
data_n = pd.read_csv('./data_no_identificada.csv')
data_n.head(1)

,Name,Netflows,First_Protocol,Second_Protocol,Third_Protocol,p1_d,p2_d,p3_d,duration,max_d,...,first_dp,second_dp,third_dp,p1_ip,p2_ip,p3_ip,p1_ib,p2_ib,p3_ib,Type
0,0-csves/0.csv,1564,TCP,UDP,ICMP,0.0,0.101,1.721,10701.93,177.706,...,53,443.0,80.0,1.0,2.0,16.0,40.0,115.5,1883.5,mine


In [18]:
data_n = data_n.drop(['Second_Protocol','Third_Protocol'], axis = 1)

## Identificar valores nulos

In [20]:
data_n.isnull().sum()

Name                  0
Netflows              0
First_Protocol        0
p1_d                  0
p2_d                  0
p3_d                  0
duration              0
max_d                 0
min_d                 0
#packets              0
Avg_bps               0
Avg_pps               0
Avg_bpp               0
#Bytes                0
#sp                   0
#dp                   0
first_sp              0
second_sp          7495
third_sp          10953
first_dp              0
second_dp          1705
third_dp           7064
p1_ip                 0
p2_ip                 0
p3_ip                 0
p1_ib                 0
p2_ib                 0
p3_ib                 0
Type                  0
dtype: int64

## Imputar valores

In [21]:
punter = pd.concat([data_n['second_sp'],data_n['third_sp'],data_n['second_dp'],data_n['third_dp']], axis = 1)
imputer = SimpleImputer(missing_values = np.nan, strategy = 'median')
values = imputer.fit_transform(punter)

In [22]:
values = pd.DataFrame(values, columns = punter.columns)

In [23]:
data_n['second_sp'] = values['second_sp']
data_n['third_sp'] = values['third_sp']
data_n['second_dp'] = values['second_dp']
data_n['third_dp'] = values['third_dp']

## OneHotEncoder

In [24]:
data_categoric = data_n.select_dtypes(['object'])
data_categoric.columns

Index(['Name', 'First_Protocol', 'Type'], dtype='object')

In [25]:
one_hot =  OneHotEncoder()
one_hot.fit_transform(data_categoric)

<37053x37058 sparse matrix of type '<class 'numpy.float64'>'
	with 111159 stored elements in Compressed Sparse Row format>

In [ ]:
dataDummy = pd.get_dummies(data_categoric)
dataDummy.head()

## Estandarización

In [ ]:
data_numeric = data_n.select_dtypes(['int64','float64'])

In [ ]:
preprocessor = ColumnTransformer([
    ('scale', StandardScaler(), data_numeric.columns), 
], remainder='passthrough')

In [ ]:
values = preprocessor.fit_transform(data_numeric)

In [ ]:
data_estandarizada = pd.DataFrame(values,columns= data_numeric.columns)
data_estandarizada.head(1)

### Concatenación de los conjuntos de datos

In [ ]:
data_n = pd.concat([data_estandarizada,dataDummy],axis = 1)
data_n.head()

In [ ]:
data_n.to_csv('dataB.csv', index = False)